<a href="https://colab.research.google.com/github/estahr/mva-project/blob/main/FinalProjectGroup5MVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A Multivariate Analysis of Baseball Player's Batting Statistics and Salaries**
**ISQS 6350**

**Group 5: Daniel Brauckman, Jordan Gussett, Bryan Loeffler and Elijah Stahr**

# Introduction


This project is based around doing multivariate analysis on baseball statistics. We use a publicly available dataset on Kaggle, involving baseball players names, various batting statistics, and salary information. Each observation (row) corresponds to a unique player in the MLB in the 2015 seawson, and each variable (column) corresponds to one of the various statistics recorded for each player. Our goal is to analyze the pay of athletes, and see if salary can be predicted or understood by various batting statistics. We also seek to understand if a player may be overvalued or undervalued based on their batting statistics relative to their salary. All of these statistical explorations have motivations in a better understanding of the game of baseball, and could even possibly be applied to more efficiently manage baseball teams. For example, if a rookie player is found to be clustered with many highly paid, established players, it may be benficial for a team to trade for the player while their salary is still relatively low.  

# Data Cleaning and Visualization

We start by allowing for R usage in Jupyter Notebook.

In [ ]:
%load_ext rpy2.ipython

We decided to use data from 2015, as it is the most current and impactful on modern salaries. We also removed data with 3 or less at bats. This is to remove any outliers due to low data. Finally, by the nature of our merge, we only retain players with recorded salaries. This is fine, because we plan on analyzing the salaries anyway. Addiationally, the batting dataset required some cleaning before merging, as playerIDs (our planned column for join) were not unique due to mid-season trades. We account for this by summing statistics between the teams, as we care about the season statistics for the player without regard for their team.

**we should put the following code in an appendix for the final project, he says to not have long code sections in the report itself**

In [ ]:
%%R

#load data sets
batting <- read.csv("https://raw.githubusercontent.com/estahr/mva-project/main/Batting.csv")
master <- read.csv("https://github.com/estahr/mva-project/raw/main/Master.csv")
salaries <- read.csv("https://raw.githubusercontent.com/estahr/mva-project/main/Salaries.csv")

#keep only 2015 data, and remove columns that we are not analyzing

batting2015 = batting[which(batting$yearID == 2015),]
batting2015 = batting2015[,-c(2:5)]
master = master[,c(1,16)]
salaries2015 = salaries[which(salaries$yearID == 2015),]
salaries2015 = salaries2015[,-c(1:3)]

# we prepare for merge by checking if playerID is unique for all datasets

print("Non unique playerID in salaries dataset:" )
print(length(salaries2015$playerID)-length(unique(salaries2015$playerID)))
print("Non unique playerID in master dataset:" )
print(nrow(master)-nrow(unique(master)))
print("Non unique playerID in batting dataset:" )
print(length(batting2015$playerID) - length(unique(batting2015$playerID)))

# We can see that we need to adjust the batting dataset to have unique playerID.
# The reason the playerID is not unique is because mid-season trades duplicate the playerID.
# We don't care what team the player played for, just their stats, so we will combine rows and add stats.


batting2015 = aggregate(batting2015[,-1], batting2015["playerID"], sum)

print("Non unique playerID in batting dataset (after sum):" )
print(length(batting2015$playerID) - length(unique(batting2015$playerID)))

#Now all playerID values are unique and we can merge

data = merge(master, batting2015)
data = merge(data, salaries2015)

print("Merged Size" )
print(dim(data))

print("Size after removing missing data (there is none):")
data = na.omit(data)
print(dim(data))

print("Size after removing players with 3 or less at bats:")
data = data[-which(data$AB <= 3),]
print(dim(data))

[1] "Non unique playerID in salaries dataset:"
[1] 0
[1] "Non unique playerID in master dataset:"
[1] 0
[1] "Non unique playerID in batting dataset:"
[1] 138
[1] "Non unique playerID in batting dataset (after sum):"
[1] 0
[1] "Merged Size"
[1] 813  20
[1] "Size after removing missing data (there is none):"
[1] 813  20
[1] "Size after removing players with 3 or less at bats:"
[1] 531  20


#